# Lecture revision

## Lecture 4

In [1]:
words = ["apple", "banana", "kiwi"]
parents = ['mum', 'dad']

def print_words(words_list):
    for p in words_list:
        print(p)
        
print_words(words)

print_words(parents)

apple
banana
kiwi
mum
dad


In [2]:
import spacy

In [3]:
nlp = spacy.load('en_core_web_md')

/Users/emma/computationalnlp/cnlp_venv/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/emma/computationalnlp/cnlp_venv/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [4]:
import spacy_transformers

In [5]:
nlp_trf = spacy.load('en_core_web_trf')

doc1 = nlp_trf("Apple shares rose on the news.")

In [6]:
doc1._.trf_data.last_hidden_layer_state[0].data.shape

(1, 768)

In [7]:
print(dir(doc1._.trf_data))

['__annotations__', '__class__', '__dataclass_fields__', '__dataclass_params__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__match_args__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'all_hidden_layer_states', 'all_outputs', 'embedding_layer', 'from_dict', 'last_hidden_layer_state', 'last_layer_only', 'num_outputs', 'to_dict']


In [8]:
# doc_split = nlp_trf('Do you really need a representation for words like miscommunication or wordly?')
# doc_split._.trf_data.tokens['input_texts']

In [9]:
from spacy.language import Language

In [10]:
import numpy as np

In [11]:
@Language.factory('tensor2attr')

class Tensor2Attr:
    def __init__(self, name, nlp):
        pass

    def __call__(self, doc):
        self.add_attributes(doc)
        return doc
    
    def add_attributes(self, doc):
        doc.user_hooks["vector"] = self.doc_tensor

        doc.user_span_hooks["vector"] = self.span_tensor
        doc.user_token_hooks["vector"] = self.token_tensor

        doc.user_hooks['similarity'] = self.get_similarity
        doc.user_span_hooks['similarity'] = self.get_similarity
        doc.user_token_hooks['similarity'] = self.get_similarity
    
    def doc_tensor(self, doc):
        return doc._.trf_data.tensors[-1].mean(axis=0)
    
    def span_tensor(self, span):
        tensor_ix = span.doc._.trf_data.align[span.start: span.end].data.flatten()
        out_dim = span.doc._.trf_data.tensors[0].shape[-1]
        tensor = span.doc._.trf_data.tensors[0].reshape(-1, out_dim)[tensor_ix]
        return tensor.mean(axis=0)
    
    def token_tensor(self, token):
        tensor_ix = token.doc._.trf_data.align[token.i].data.flatten()
        out_dim = token.doc._.trf_data.tensors[0].shape[-1]
        tensor = token.doc._.trf_data.tensors[0].reshape(-1, out_dim)[tensor_ix]
        return tensor.mean(axis=0)
    
    def get_similarity(self, doc1, doc2):
        return np.dot(doc1.vector, doc2.vector) / (doc1.vector_norm * doc2.vector_norm)
        

In [12]:
nlp_trf.add_pipe('tensor2attr')

nlp_trf.pipeline

[('transformer',
  <spacy_curated_transformers.pipeline.transformer.CuratedTransformer at 0x153e25f70>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x153e26270>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x1479a8dd0>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x153e6cbd0>),
 ('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x153dd4750>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x1479a89e0>),
 ('tensor2attr', <__main__.Tensor2Attr at 0x157a30950>)]

In [13]:
doc1 = nlp_trf("Apple shares rose on the news.")
doc2 = nlp_trf("I really enjoyed this delicious apple.")

print(doc1[0])

print(doc2[5])

# print(doc1[0].similarity(doc2[5]))

Apple
apple


In [14]:
from transformers import pipeline

pipe = pipeline("text-generation")

def data():
    while True: 
        yield "this is a test"

for out in pipe(data()):
    print(out)

No model was supplied, defaulted to gpt2 and revision 6c0e608 (https://huggingface.co/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "this is a test and i don't understand how you could make it work for everyone, so i asked for some advice on how to test your software but they told me it's not that important so i was happy there!\n\nOk so the"}]


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'this is a test for null on both of the two implementations of the function that we are testing.]\n\n<script type="text/javascript"> var t = document.createElement(\'script\'); var r = Document.createElement(\'script\');'}]


KeyboardInterrupt: 